# Testing Universal Sentence Encoder for semantic similarity

* https://tfhub.dev/google/universal-sentence-encoder/4
* https://www.tensorflow.org/hub/tutorials/semantic_similarity_with_tf_hub_universal_encoder


In [ ]:
!pip install tensorflow
!pip install tensorflow_hub

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf

import tensorflow_hub as hub

In [4]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)


module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


## 1. Load the sts-benchmark data and remove lines that contain errors.

In [5]:
# Remove "warn_bad_lines=False" to print the lines that have errors.
train_df = pd.read_csv('stsbenchmark/sts-train.csv', sep='\t', engine='python', header=None, encoding='utf-8', error_bad_lines=False, warn_bad_lines=False)

## 2. A quick look at the dataset we are using

In [6]:
print(train_df.loc[0])
print('\n')
print(train_df.loc[45])

train_df.head()

0                  main-captions
1                         MSRvid
2                       2012test
3                              1
4                              5
5         A plane is taking off.
6    An air plane is taking off.
Name: 0, dtype: object


0                     main-captions
1                            MSRvid
2                          2012test
3                                68
4                                 1
5       A man is playing the piano.
6    A woman is playing the violin.
Name: 45, dtype: object


,0,1,2,3,4,5,6
0,main-captions,MSRvid,2012test,1,5.00,A plane is taking off.,An air plane is taking off.
1,main-captions,MSRvid,2012test,4,3.80,A man is playing a large flute.,A man is playing a flute.
2,main-captions,MSRvid,2012test,5,3.80,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...
3,main-captions,MSRvid,2012test,6,2.60,Three men are playing chess.,Two men are playing chess.
4,main-captions,MSRvid,2012test,9,4.25,A man is playing the cello.,A man seated is playing the cello.


## 3. Comparing two sentence paires with Universal Sentence Encoder as an example

In [14]:
s1 = [train_df.loc[0][5]]
s2 = [train_df.loc[0][6]]
s3 = [train_df.loc[45][5]]
s4 = [train_df.loc[45][6]]

print(f's1 = {s1}')
print(f's2 = {s2}')
print('\n')
print(f's3 = {s3}')
print(f's4 = {s4}')

s1 = ['A plane is taking off.']
s2 = ['An air plane is taking off.']


s3 = ['A man is playing the piano.']
s4 = ['A woman is playing the violin.']


In [15]:
from scipy.spatial import distance

s1_embedding = embed(s1)
s2_embedding = embed(s2)
s3_embedding = embed(s3)
s4_embedding = embed(s4)

print(f's1 vs s2 = {distance.cosine(s1_embedding,s2_embedding)}')
print(f'Human score = {train_df.loc[0][4]}')
print(f'USE Score = {round((1-distance.cosine(s1_embedding,s2_embedding))*5,1)}')

print(f's3 vs s4 = {distance.cosine(s3_embedding,s4_embedding)}')
print(f'Human score = {train_df.loc[45][4]}')
print(f'USE Score = {round((1-distance.cosine(s3_embedding,s4_embedding))*5,1)}')

print(f's1 vs s3 = {distance.cosine(s1_embedding,s3_embedding)}')
print(f's1 vs s4 = {distance.cosine(s1_embedding,s4_embedding)}')

s1 vs s2 = 0.07879352569580078
Human score = 5.0
USE Score = 4.6
s3 vs s4 = 0.4041421413421631
Human score = 1.0
USE Score = 3.0
s1 vs s3 = 0.7778528481721878
s1 vs s4 = 0.8449013531208038


## 4. Getting the human scores and the SentenceBERT scores and comparing them